# Multiscale MILP with Materials and Emission

This is a continuation of 'One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming Example' [Example 2]. 

We will be adding the following considerations: 

1. material for the establishment of operations 
2. global warming potential induced as a sum of our decisions and choices 

## See Example 2

In [1]:
from energia import *

m = Model('example3')
m.q = Periods()
m.y = 4 * m.q


In [2]:
m.usd = Currency()
m.declare(Resource, ['power', 'wind', 'solar'])
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49

m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

--- General Resource Balance for solar in (l, q): initializing constraint, adding consume(solar, l, q)
    Completed in 0.0004639625549316406 seconds
--- Binding consume in domain (solar, l, q)
    Completed in 0.00016498565673828125 seconds
--- General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
    Completed in 0.0009357929229736328 seconds
--- Binding consume in domain (wind, l, y)
    Completed in 0.00013113021850585938 seconds
--- General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
    Completed in 0.0001881122589111328 seconds
--- Binding release in domain (power, l, q)
    Completed in 0.00025010108947753906 seconds
--- Binding capacity in domain (wf, l, y)
    Completed in 0.0002219676971435547 seconds
--- Binding capacity in domain (wf, l, y)
    Completed in 0.0002071857452392578 seconds
--- Binding operate in domain (wf, l, q)
    Completed in 0.0006358623504638672 seconds
--- Mapping

## Indicator Stream 

Indicators scale the impact of some decision or flow and project it only a common dimension allowing the calculation of overall impact as a single metric value

In this case, we consider global warming potential (GWP).

In [3]:
m.gwp = Environ()

Now say that we want to consider the impact of setting up operations (specific to the act of construction)

These can be provided as calculations 

In [4]:
m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that impact streams are mapped across scales but not balanced, unless a negative (relatively) impact is also assessed! In the case of emission, this would be abatement.

## Material Stream

Note that all the objects found in energia.components.commodity.misc are still resources (Land, Material, etc.)

This will allow for the modeling of an expansive type of processes.

Examples:

1. land clearance to transform agricultural land into industrial, or land remediation 
2. resource intense material recycling 
3. some resource flowing being treated as emission flows 

The possibilities are vast... The advanced modeler may prefer to use Resource() for everything [see Resource Task Network (RTN) Framework]

In the following example, we consider a limit to cement consumption, as also an expense and gwp impact associated with it

In [5]:
m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17
m.cement.consume[m.gwp.emit] == 0.9

--- General Resource Balance for cement in (l, y): initializing constraint, adding consume(cement, l, y)
    Completed in 0.00032591819763183594 seconds
--- Binding consume in domain (cement, l, y)
    Completed in 0.00011587142944335938 seconds


Next, we provide details of the use of cement across all operations 

In [6]:
m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

--- General Resource Balance for cement in (l, y): adding use(cement, l, y, capacity, wf)
    Completed in 0.00017905235290527344 seconds
--- General Resource Balance for cement in (l, y): adding use(cement, l, y, capacity, pv)
    Completed in 0.000164031982421875 seconds
--- General Resource Balance for cement in (l, y): adding use(cement, l, y, invcapacity, power.lii)
    Completed in 0.0001399517059326172 seconds


Material use is summed up across all operations, and adheres to an upper bound in terms of consumption. 

In [7]:
m.cement.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

GWP impact is now the sum of impact from material use as well as constuction 

In [8]:
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Locate Operations

In [9]:
m.network.locate(m.wf, m.pv, m.lii)

--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
    Completed in 0.0004398822784423828 seconds
--- General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
    Completed in 0.00013589859008789062 seconds
--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
    Completed in 0.00019812583923339844 seconds
--- General Resource Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
    Completed in 0.0001990795135498047 seconds
--- Assuming  power.lii inventory capacity is unbounded in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, q)
--- Mapping inventory across time from (power.lii, l, q) to (power.lii, l, y)
--- Creating map to (power.lii, l, y). Mapping inventory: from (power.lii, l, q) to (power.lii, l, y)
    Completed in 0.0002181529998779297 seconds
---

## Optimization 

Let us minimize GWP this time. Note that m.usd.spend.opt() can still be used!

In [10]:
m.gwp.emit.opt()

--- Creating map to (gwp, l, y). Mapping emit: from (gwp, l, y, capacity, wf) to (gwp, l, y)
    Completed in 0.00043487548828125 seconds
--- Mapping emit: from (gwp, l, y, capacity, pv) to (gwp, l, y)
    Completed in 0.00010585784912109375 seconds
--- Mapping emit: from (gwp, l, y, invcapacity, power.lii) to (gwp, l, y)
    Completed in 9.298324584960938e-05 seconds
--- Mapping emit: from (gwp, l, y, consume, cement) to (gwp, l, y)
    Completed in 0.0007197856903076172 seconds
--- Generating Program(example3).mps
--- Creating gurobi model for Program(example3)
Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-16
Read MPS format model from file Program(example3).mps
Reading time = 0.01 seconds
PROGRAM(EXAMPLE3): 95 rows, 87 columns, 217 nonzeros
--- Optimizing Program(example3) using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 phy

In [11]:
len(m.variables)

87

In [12]:
m.show()

# Mathematical Program for Program(example3)

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Function Sets

<IPython.core.display.Math object>

In [13]:
m.maps

{operate: {(wf, l, y): [(wf, l, q)],
  (pv, l, y): [(pv, l, q)],
  (lii.charge, l, y): [(lii.charge, l, q)],
  (lii.discharge, l, y): [(lii.discharge, l, q)]},
 inventory: {(power.lii, l, y): [(power.lii, l, q)]},
 emit: {(gwp, l, y): [(gwp, l, y, capacity, wf),
   (gwp, l, y, capacity, pv),
   (gwp, l, y, invcapacity, power.lii),
   (gwp, l, y, consume, cement)]}}

In [18]:
for c in m.constraints:
    print(c)

1.0×consume[0]-expend[1]=0
1.0×consume[1]-expend[2]=0
1.0×consume[2]-expend[3]=0
1.0×consume[3]-expend[4]=0
consume[0]-100.0<=0
consume[1]-100.0<=0
consume[2]-100.0<=0
consume[3]-100.0<=0
1.0×consume[4]-expend[0]=0
consume[4]-400.0<=0
-1.0×release[0]+produce[0]+produce[4]-expend[5]+produce[12]=0
-1.0×release[1]+produce[1]+produce[5]-expend[6]+produce[13]=0
-1.0×release[2]+produce[2]+produce[6]-expend[7]+produce[14]=0
-1.0×release[3]+produce[3]+produce[7]-expend[8]+produce[15]=0
-1.0×release[0]+134.99999999999997<=0
-1.0×release[1]+157.49999999999997<=0
-1.0×release[2]+180.0<=0
-1.0×release[3]+67.49999999999999<=0
capacity[0]-100.0×x_capacity[0]<=0
10.0×x_capacity[0]-capacity[0]<=0
operate[0]-1.0×capacity[0]<=0
operate[1]-0.888888888888889×capacity[0]<=0
operate[2]-0.5555555555555556×capacity[0]<=0
operate[3]-0.7777777777777777×capacity[0]<=0
spend[0]-993991.0×capacity[0]=0
operate[4]-sigma(operate[0]((wf, l, q[0])),q)=0
spend[1]-49.0×operate[4]=0
capacity[1]-100.0×x_capacity[1]<=0
10.0

In [11]:
m.show()

# Mathematical Program for Program(example3)

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Function Sets

<IPython.core.display.Math object>

In [ ]:
m.power.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.sol(False)

# Solution for example3

<br><br>

## Objective

<IPython.core.display.Math object>

<br><br>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>